In [50]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/dataq/main/mtcars.csv',index_col=0)
df.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


#### # 1
mtcars 데이터셋(mtcars.csv)의 qsec 컬럼을 최소 최대 척도(min-max scale)로 변환한 후 0.5보다 큰 값을 가지는 레코드 수를 구하시오

In [51]:
#MinMaxScalar -> 0.5이상 레코드 수 
qsec = df['qsec']

In [52]:
from sklearn.preprocessing import minmax_scale
qsec_minmax_scale = minmax_scale(qsec)
qsec_minmax_scale

array([0.23333333, 0.3       , 0.48928571, 0.58809524, 0.3       ,
       0.68095238, 0.15952381, 0.6547619 , 1.        , 0.45238095,
       0.52380952, 0.3452381 , 0.36904762, 0.41666667, 0.41428571,
       0.3952381 , 0.34761905, 0.59166667, 0.47857143, 0.64285714,
       0.65595238, 0.28214286, 0.33333333, 0.10833333, 0.30357143,
       0.52380952, 0.26190476, 0.28571429, 0.        , 0.11904762,
       0.01190476, 0.48809524])

In [53]:
qsec_count = len(qsec_minmax_scale[qsec_minmax_scale>0.5])

In [54]:
print(qsec_count)

9


### # 2
mtcars 데이터셋(mtcars.csv)의 qsec 컬럼을 표준정규분포 데이터 표준화 (standardization) 변환 후 최대, 최소값을 각각 구하시오.

In [55]:
qsec = qsec.values.reshape(-1,1) # array 1차원배열 -> 1차원으로 만들기 

In [56]:
from sklearn. preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(qsec)
qsec_scaled = scaler.transform(qsec)

In [60]:
print("최소값 :", min(qsec_scaled), "\n최대값 :", max(qsec_scaled))

최소값 : [-1.90399635] 
최대값 : [2.87198554]


In [62]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(qsec)
qsec_scaled = scaler.transform(qsec)
print("최소값 :", min(qsec_scaled), "\n최대값 :", max(qsec_scaled))

최소값 : [0.] 
최대값 : [1.]


### # 3
*mtcars 데이터셋(mtcars.csv)의 wt 컬럼의 이상치(IQR 1.5 외부에 존재하는)값들을 outlier 변수에 저장하라

In [75]:
Q1 = df.wt.quantile(0.25)
Q3 = df.wt.quantile(0.75)
IQR = Q3-Q1
outlier = df.wt[(df.wt<=Q1-1.5*IQR)|(df.wt>=Q3+1.5*IQR)]

In [84]:
np.array(outlier)

array([5.25 , 5.424, 5.345])

### # 4
mtcars 데이터셋에서 mpg변수와 나머지 변수들의 상관계수를 구하여 다음과 같이 내림차순 정렬하여 표현하라

In [118]:
df.corr()[['mpg']][1:].sort_values('mpg', ascending=False)

,mpg
drat,0.681172
vs,0.664039
am,0.599832
gear,0.480285
qsec,0.418684
carb,-0.550925
hp,-0.776168
disp,-0.847551
cyl,-0.852162
wt,-0.867659


In [139]:
import scipy.stats as stats
stats.pearsonr(df.mpg,df.drat)

(0.6811719078067493, 1.7762399287524147e-05)

### # 5
mtcars 데이터셋에서 mpg변수를 제외하고 데이터 정규화 (standardscaler) 과정을 진행한 이후 PCA를 통해 변수 축소를 하려한다. 누적설명 분산량이 92%를 넘기기 위해서는 몇개의 주성분을 선택해야하는지 설명하라

In [210]:
df_except_mpg = df.iloc[:,1:] #iloc[행의위치, 열의위치]
df_scaled = scaler.transform(df_except_mpg)

In [211]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
df_pca = pca.fit_transform(df_scaled)

In [204]:
print(pca.explained_variance_ratio_)

[9.27327666e-01 7.24932662e-02 8.62022996e-05 4.64428194e-05
 2.93759754e-05 5.30086229e-06 4.16685850e-06 3.13315551e-06
 2.45448661e-06 1.99131730e-06]


In [212]:
componentsNum=10
pca = PCA(n_components=componentsNum) 
printcipalComponents = pca.fit_transform(df_scaled)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['component'+str(x) for x in range(componentsNum)])

componentDf = pd.DataFrame(pca.explained_variance_ratio_,columns=['cumsumVariance']).cumsum().reset_index()
componentDf['index'] +=1
componentDf=componentDf.rename(columns={'index':'componentsCount'})

In [213]:
componentDf

,componentsCount,cumsumVariance
0,1,0.927328
1,2,0.999821
2,3,0.999907
3,4,0.999954
4,5,0.999983
5,6,0.999988
6,7,0.999992
7,8,0.999996
8,9,0.999998
9,10,1.000000


In [218]:
pca = PCA(n_components = 0.92)

pca_df = pca.fit_transform(df_scaled)

print('주성분 수: ', pca.n_components_)

print('주성분 비율: ', np.sum(pca.explained_variance_ratio_))

주성분 수:  1
주성분 비율:  0.9273276660690598
